# Kalman Filter Hainich
> Kalman filter model on Hainich data using PyTorch 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from meteo_imp.data import hai, units
from meteo_imp.data_preparation import MeteoDataTest
from meteo_imp.kalman.imputation import KalmanImputation
from meteo_imp.kalman.model import *
from ipywidgets import interact, interact_manual, IntSlider

In [ ]:
hai

,TA,SW_IN,VPD
TIMESTAMP_END,,,
200001011712.0,-0.60,0.0,0.222
200001011712.0,-0.65,0.0,0.122
200001011712.0,-0.58,0.0,0.090
200001011712.0,-0.51,0.0,0.110
200001011712.0,-0.49,0.0,0.102
...,...,...,...
200001044480.0,4.74,0.0,1.191
200001044480.0,4.75,0.0,1.057
200001044480.0,4.76,0.0,0.935


In [ ]:
data = MeteoDataTest(hai).add_gap(10, ['TA', 'SW_IN', 'VPD'], 30)

In [ ]:
data.data

,TA,SW_IN,VPD
TIMESTAMP_END,,,
200001011712.0,-0.60,0.0,0.222
200001011712.0,-0.65,0.0,0.122
200001011712.0,-0.58,0.0,0.090
200001011712.0,-0.51,0.0,0.110
200001011712.0,-0.49,0.0,0.102
...,...,...,...
200001044480.0,4.74,0.0,1.191
200001044480.0,4.75,0.0,1.057
200001044480.0,4.76,0.0,0.935


In [ ]:
import numpy as np

In [ ]:
imp = KalmanImputation(data.data)

In [ ]:
imp.fit(n_iter = 50)

  0%|          | 0/50 [00:00<?, ?it/s]

ValueError: Expected parameter covariance_matrix (Tensor of shape (3, 3)) of distribution MultivariateNormal(loc: torch.Size([3]), covariance_matrix: torch.Size([3, 3])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[ 0.6063, -0.3421, -0.3056],
        [-0.3361,  0.6454, -0.3535],
        [-0.3007, -0.3570,  0.5838]], grad_fn=<ExpandBackward0>)

In [ ]:
imp.model.plot_loss()

In [ ]:
imp.impute()

In [ ]:
data.data.columns

In [ ]:
res = imp.to_result(data.data_compl_tidy, var_names= data.data.columns, units=units, pred_all=True)

In [ ]:
res.display_results()

## Interactive

In [ ]:
def gap2res(var_sel, gap_len, gap_start, n_iter):
    data = MeteoDataTest(hai).add_gap(gap_len, var_sel, gap_start)
    return KalmanImputation(data.data).fit(n_iter=n_iter).to_result(data.data_compl_tidy, var_names= data.data.columns, units=units, pred_all=True)

In [ ]:
var_sel = data.data.columns

In [ ]:
%time gap2res(var_sel, 10, 10, 5)

In [ ]:
gaps = [2, 5, 7, 10, 20, 30, 50, 100]
gap_starts = [0, 30, 60, 90]

In [ ]:
#| include: false
@interact_manual(TA=True, SW_IN=True, VPD=True,
          gap_len=IntSlider(10, 1,100),
          gap_start=IntSlider(30, 1,100),
          n_iter = (10, 100))
def show_diff_gaps_res(TA, SW_IN, VPD, gap_len, gap_start, n_iter):
    var_sel = []
    if TA: var_sel.append('TA')
    if SW_IN: var_sel.append('SW_IN')
    if VPD: var_sel.append('VPD')
    var_sel = (*var_sel,)
    gap2res(var_sel, gap_len, gap_start, n_iter).display_results()
    